# 实验 3.4：文本对抗攻击

## 🎯 学习目标

完成本实验后，你将能够：
- ✅ 理解文本对抗攻击与图像攻击的本质区别
- ✅ 实现基于同义词替换的文本对抗攻击
- ✅ 分析词重要性并设计针对性攻击策略
- ✅ 认识文本对抗攻击在真实场景中的威胁

## 📚 前置知识

- 了解自然语言处理（NLP）基础
- 理解文本分类任务（如情感分析）
- 完成前三个实验（图像对抗攻击）

## 🖥️ 实验环境

| 项目 | 说明 |
|------|------|
| 平台 | 腾讯 Cloud Studio |
| GPU | NVIDIA Tesla T4（16GB）|
| 模型 | hfl/chinese-macbert-base (中文BERT) |
| 任务 | 中文情感分析 |

## ⏱️ 预计时间：30 分钟

## 📝 填空说明

本实验共 **5 个填空练习**，难度：⭐⭐⭐☆☆

## 核心概念回顾

### 文本 vs 图像对抗攻击的区别

| 维度 | 图像攻击 | 文本攻击 |
|------|---------|---------|
| **数据性质** | 连续（像素值） | 离散（词语） |
| **扰动方式** | 微小像素改变 | 词语替换/插入/删除 |
| **梯度可用性** | 可直接计算梯度 | 梯度不可直接应用 |
| **语义保持** | 视觉相似 | 语义相似 |
| **检测难度** | 人眼难察觉 | 容易产生语法错误 |

### 文本对抗攻击策略

**1. 字符级攻击**
- 插入/删除字符
- 同形字替换（如 `0` → `O`）
- 打字错误模拟

**2. 词级攻击**
- 同义词替换（最常用）
- 词序调整
- 重要词删除

**3. 句子级攻击**
- 添加无关句子
- 改写句子结构

### 同义词替换攻击流程

```
1. 分析原始文本，获取模型预测
2. 计算每个词的重要性（影响预测的程度）
3. 优先替换重要性高的词
4. 从同义词库中选择替换词
5. 验证替换后的预测是否改变
6. 重复直到攻击成功或无可替换词
```

**关键约束**：
- 语义保持：替换后文本意思不变
- 语法正确：不产生语法错误
- 人类可读：人类能理解文本内容

## 第一部分：环境准备与模型加载

In [ ]:
# 安装依赖
import sys
!{sys.executable} -m pip install transformers torch -q

# 导入必要的库
import torch
import numpy as np
import matplotlib.pyplot as plt
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import warnings
warnings.filterwarnings('ignore')

# 设置中文显示
plt.rcParams['font.sans-serif'] = ['SimHei', 'Arial Unicode MS', 'DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False

# 设置随机种子
torch.manual_seed(42)
np.random.seed(42)

print("=" * 50)
print("🔧 环境准备完成")
print("=" * 50)

In [ ]:
# 加载中文情感分析模型
print("=" * 50)
print("📦 加载中文BERT模型")
print("=" * 50)
print("正在下载模型（首次运行需要几分钟）...\n")

# 使用京东商品评论情感分析模型（已训练好的中文情感分类器）
model_name = "uer/roberta-base-finetuned-jd-binary-chinese"

print("1️⃣ 加载分词器...")
tokenizer = AutoTokenizer.from_pretrained(model_name)
print("   ✓ Tokenizer 加载完成")

print("\n2️⃣ 加载情感分类模型...")
model = AutoModelForSequenceClassification.from_pretrained(model_name)
model.eval()  # 设置为评估模式

param_count = sum(p.numel() for p in model.parameters()) / 1e6
print(f"   ✓ 模型加载完成 (~{param_count:.1f}M 参数)")

print("\n" + "=" * 50)
print("✓ 模型加载成功！")
print(f"  模型: RoBERTa-Base (中文)")
print(f"  任务: 二分类情感分析 (正面/负面)")
print(f"  训练数据: 京东商品评论")
print("=" * 50)

In [ ]:
# 定义中文同义词词典
# 基于常见电商评论场景的同义词替换词库

SYNONYMS = {
    # 正面词汇
    "好": ["棒", "佳", "优", "赞", "妙"],
    "喜欢": ["爱", "青睐", "偏爱", "钟爱", "中意"],
    "优秀": ["出色", "杰出", "卓越", "优良", "上乘"],
    "满意": ["称心", "如意", "舒心", "惬意", "开心"],
    "推荐": ["建议", "介绍", "举荐", "力荐", "安利"],
    "快": ["迅速", "敏捷", "飞快", "神速", "快捷"],
    "便宜": ["实惠", "划算", "低价", "优惠", "亲民"],
    "漂亮": ["美丽", "好看", "靓丽", "精美", "美观"],
    "不错": ["可以", "挺好", "尚可", "还行", "合格"],
    "质量": ["品质", "质地", "档次", "水平", "水准"],
    
    # 负面词汇
    "差": ["糟", "烂", "劣", "次", "孬"],
    "讨厌": ["厌恶", "反感", "嫌弃", "憎恨", "恶心"],
    "失望": ["沮丧", "灰心", "气馁", "心寒", "泄气"],
    "慢": ["缓慢", "迟缓", "磨蹭", "拖拉", "迟钝"],
    "贵": ["昂贵", "高价", "天价", "奢侈", "死贵"],
    "难用": ["难操作", "不好用", "复杂", "麻烦", "费劲"],
    "后悔": ["遗憾", "懊悔", "悔恨", "痛心", "悔不当初"],
    
    # 中性词汇
    "产品": ["商品", "货物", "物品", "东西", "物件"],
    "购买": ["买", "选购", "入手", "下单", "采购"],
    "使用": ["用", "运用", "利用", "采用", "应用"],
    "发货": ["送货", "配送", "物流", "快递", "寄送"],
    "服务": ["态度", "接待", "对待", "待遇", "款待"],
}

def get_synonyms(word):
    """
    获取词的同义词列表
    
    参数:
        word: 要查找同义词的词
    
    返回:
        synonyms: 同义词列表
    """
    return SYNONYMS.get(word, [])

print(f"📚 同义词词典统计:")
print(f"  词条数量: {len(SYNONYMS)}")
print(f"  总同义词数: {sum(len(v) for v in SYNONYMS.values())}")
print(f"\n示例:")
print(f"  '好' 的同义词: {get_synonyms('好')}")
print(f"  '喜欢' 的同义词: {get_synonyms('喜欢')}")
print(f"  '差' 的同义词: {get_synonyms('差')}")

In [ ]:
# 定义情感分析辅助函数
def analyze_sentiment(text):
    """
    分析文本情感
    
    参数:
        text: 输入文本
    
    返回:
        label: 情感标签 ("正面" 或 "负面")
        confidence: 预测置信度
    """
    # 分词和编码
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=128)
    
    # 模型推理
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        probs = torch.softmax(logits, dim=1)[0]
    
    # 标签映射: 0=负面, 1=正面
    pred_label = torch.argmax(probs).item()
    confidence = probs[pred_label].item()
    
    label = "正面" if pred_label == 1 else "负面"
    return label, confidence

def get_positive_score(text):
    """
    获取正面情感的置信度分数
    
    参数:
        text: 输入文本
    
    返回:
        score: 正面情感置信度 [0, 1]
    """
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=128)
    with torch.no_grad():
        outputs = model(**inputs)
        probs = torch.softmax(outputs.logits, dim=1)[0]
    return probs[1].item()  # 索引1对应正面

# 测试情感分析功能
test_texts = [
    "这个产品非常好，我很喜欢！",
    "质量太差了，完全不推荐购买。",
    "还可以吧，价格有点贵。",
    "发货速度快，服务态度好，满意！",
    "用了几天就坏了，真是失望。"
]

print("=" * 60)
print("🧪 情感分析功能测试")
print("=" * 60)
for text in test_texts:
    label, score = analyze_sentiment(text)
    color_code = '\033[92m' if label == "正面" else '\033[91m'
    reset_code = '\033[0m'
    print(f"  {text[:30]:<30} → {label} ({score:.2%})")

## 第二部分：词重要性分析

了解哪些词对模型预测影响最大，是设计高效攻击的关键。

In [ ]:
def compute_word_importance(text):
    """
    计算文本中每个字符对预测结果的重要性
    
    方法：删除法 (Leave-One-Out)
    - 删除某个字符，观察预测置信度的变化
    - 变化越大，说明该字符越重要
    
    参数:
        text: 输入文本
    
    返回:
        importance_scores: [(字符, 重要性分数), ...]
    """
    # 获取原始预测分数
    original_score = get_positive_score(text)
    
    # 简单分词（按字符）
    chars = list(text)
    importance_scores = []
    
    for i, char in enumerate(chars):
        # 跳过标点符号
        if char in ['，', '。', '！', '？', ' ', '、', '；']:
            importance_scores.append((char, 0))
            continue
        
        # ========== 填空 1：计算删除该字后的文本 ==========
        # 
        # 🎯 任务：创建删除第 i 个字符后的新文本
        # 
        # 💡 提示：
        #   - 拼接 i 之前和 i 之后的字符
        #   - chars[:i] 是前面的字符
        #   - chars[i+1:] 是后面的字符
        #   - 使用 ''.join() 拼接成字符串
        # 
        # 难度：⭐⭐☆☆☆
        
        chars_without_i = ___________  # 期望：chars[:i] + chars[i+1:]
        modified_text = ''.join(chars_without_i)
        
        if not modified_text.strip():  # 空文本跳过
            importance_scores.append((char, 0))
            continue
        
        # 计算删除该字后的情感分数
        modified_score = get_positive_score(modified_text)
        
        # 重要性 = 删除前后分数变化的绝对值
        importance = abs(original_score - modified_score)
        
        importance_scores.append((char, importance))
    
    return importance_scores

# 测试词重要性分析
test_sentence = "这个产品非常好，我很喜欢！"

print("=" * 60)
print("🔍 词重要性分析")
print("=" * 60)
print(f"测试文本: '{test_sentence}'")

orig_label, orig_conf = analyze_sentiment(test_sentence)
print(f"原始预测: {orig_label} ({orig_conf:.2%})\n")

importance = compute_word_importance(test_sentence)

# 按重要性排序并显示前10
sorted_importance = sorted(importance, key=lambda x: x[1], reverse=True)
print("字符重要性排名（前10）:")
print(f"{'排名':<6} {'字符':<10} {'重要性分数':<15}")
print("-" * 40)
for rank, (char, score) in enumerate(sorted_importance[:10], 1):
    if score > 0:
        print(f"{rank:<6} {char:<10} {score:<15.4f}")

In [ ]:
# 可视化词重要性
chars = [c for c, _ in importance]
scores = [s for _, s in importance]

plt.figure(figsize=(16, 5))

# 根据重要性设置颜色
colors = []
for s in scores:
    if s > 0.05:
        colors.append('red')      # 高重要性
    elif s > 0.02:
        colors.append('orange')   # 中等重要性
    else:
        colors.append('steelblue')  # 低重要性

plt.bar(range(len(chars)), scores, color=colors)
plt.xticks(range(len(chars)), chars, fontsize=13)
plt.xlabel('字符位置', fontsize=12)
plt.ylabel('重要性分数', fontsize=12)
plt.title(f'字符重要性热力图\n原文: "{test_sentence}"', fontsize=14, fontweight='bold')
plt.axhline(y=0.05, color='red', linestyle='--', alpha=0.5, label='高重要性阈值')
plt.axhline(y=0.02, color='orange', linestyle='--', alpha=0.5, label='中等重要性阈值')
plt.legend()
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

print("\n💡 攻击策略提示:")
print("   红色字符 = 高重要性，优先攻击目标")
print("   橙色字符 = 中等重要性，次要攻击目标")
print("   蓝色字符 = 低重要性，替换效果不明显")

## 第三部分：实现同义词替换攻击

基于词重要性，我们设计一个贪心策略：优先替换重要性高的词。

In [ ]:
def synonym_attack(text, max_replacements=5):
    """
    同义词替换对抗攻击
    
    策略：
    1. 遍历同义词词典中的所有词
    2. 如果词在文本中，尝试替换为同义词
    3. 选择使置信度变化最大的替换
    4. 重复直到攻击成功或达到最大替换次数
    
    参数:
        text: 原始文本
        max_replacements: 最大替换次数
    
    返回:
        adv_text: 对抗文本
        success: 是否攻击成功
        replacements: 替换历史 [(原词, 替换词), ...]
    """
    original_label, original_conf = analyze_sentiment(text)
    current_text = text
    replacements = []
    
    # 遍历同义词词典
    for word, synonyms in SYNONYMS.items():
        # 检查词是否在当前文本中
        if word not in current_text:
            continue
        
        # 尝试每个同义词，找到最佳替换
        best_synonym = None
        best_score_change = 0
        
        for synonym in synonyms:
            # ========== 填空 2：创建替换后的文本 ==========
            # 
            # 🎯 任务：将文本中的目标词替换为同义词
            # 
            # 💡 提示：
            #   - 使用字符串的 replace 方法
            #   - current_text.replace(old, new, count)
            #   - count=1 表示只替换第一个匹配
            # 
            # 难度：⭐☆☆☆☆
            
            test_text = ___________  # 期望：current_text.replace(word, synonym, 1)
            
            new_label, new_conf = analyze_sentiment(test_text)
            
            # 如果攻击成功（标签改变），直接返回
            if new_label != original_label:
                replacements.append((word, synonym))
                return test_text, True, replacements
            
            # 计算置信度变化（希望降低原标签的置信度）
            if original_label == "正面":
                score_change = original_conf - new_conf
            else:
                score_change = new_conf - original_conf
            
            # 记录最佳替换
            if score_change > best_score_change:
                best_score_change = score_change
                best_synonym = synonym
        
        # 应用最佳替换（即使没有改变标签，也尝试降低置信度）
        if best_synonym and best_score_change > 0.01:
            replacements.append((word, best_synonym))
            current_text = current_text.replace(word, best_synonym, 1)
            
            # 检查是否攻击成功
            new_label, _ = analyze_sentiment(current_text)
            if new_label != original_label:
                return current_text, True, replacements
            
            # 达到最大替换次数
            if len(replacements) >= max_replacements:
                break
    
    # 攻击失败，返回当前最佳结果
    final_label, _ = analyze_sentiment(current_text)
    success = (final_label != original_label)
    return current_text, success, replacements

print("✓ 同义词替换攻击函数定义完成！")

In [ ]:
# 测试同义词替换攻击
test_sentences = [
    "这个产品非常好，我很喜欢！",
    "质量优秀，价格便宜，强烈推荐购买！",
    "发货快，产品漂亮，非常满意。",
    "太差了，质量很差，不推荐。",
    "用了几天就坏了，很失望。",
]

print("=" * 75)
print("⚔️  同义词替换攻击测试")
print("=" * 75)

for i, sentence in enumerate(test_sentences, 1):
    orig_label, orig_conf = analyze_sentiment(sentence)
    
    # ========== 填空 3：执行同义词替换攻击 ==========
    # 
    # 🎯 任务：对每个测试句子执行攻击
    # 
    # 💡 提示：
    #   - 调用 synonym_attack 函数
    #   - 参数是当前的 sentence
    #   - 返回三个值：对抗文本、成功标志、替换历史
    # 
    # 难度：⭐☆☆☆☆
    
    adv_text, success, replacements = ___________  # 期望：synonym_attack(sentence)
    
    adv_label, adv_conf = analyze_sentiment(adv_text)
    
    print(f"\n【测试 {i}】")
    print(f"  原文: {sentence}")
    print(f"  原始: {orig_label} ({orig_conf:.2%})")
    
    if replacements:
        print(f"  替换: {' → '.join([f'{o}→{n}' for o, n in replacements])}")
    else:
        print(f"  替换: 无")
    
    print(f"  对抗: {adv_text}")
    print(f"  结果: {adv_label} ({adv_conf:.2%})", end=" ")
    
    if success:
        print("- ✓ 攻击成功！")
    else:
        print("- ✗ 攻击失败")

print("=" * 75)

### 🤔 思考一下

1. **攻击难度**：文本攻击比图像攻击更容易还是更难？为什么？
2. **语义保持**：同义词替换是否完美保持了原文的语义？
3. **防御策略**：如果你是防御者，如何检测或防御这种攻击？

## 第四部分：批量测试与效果分析

在更多样本上测试攻击成功率，分析攻击效果。

In [ ]:
# 批量测试攻击成功率
test_samples = [
    "这款手机非常好用，我很喜欢。",
    "产品质量优秀，物流也很快。",
    "服务态度好，产品漂亮，满意！",
    "价格便宜，性价比高，推荐购买。",
    "物超所值，强烈推荐！",
    "太差了，质量很差，失望。",
    "发货慢，产品难用，不满意。",
    "很失望，完全不推荐。",
    "贵而且质量差，后悔买了。",
    "用了一天就坏了，真差。",
]

# ========== 填空 4：批量测试并统计结果 ==========
# 
# 🎯 任务：对所有样本执行攻击并统计成功率
# 
# 💡 提示：
#   - 遍历 test_samples
#   - 对每个样本调用 synonym_attack
#   - 统计成功次数和平均替换词数
# 
# 难度：⭐⭐☆☆☆

print("=" * 70)
print("📊 批量攻击测试")
print("=" * 70)
print(f"{'样本':<35} {'替换词数':<12} {'攻击结果':<12}")
print("-" * 70)

success_count = 0
total_replacements = 0
successful_replacements = []

for sample in test_samples:
    adv_text, success, replacements = ___________  # 期望：synonym_attack(sample)
    
    if success:
        success_count += 1
        num_replacements = len(replacements)
        total_replacements += num_replacements
        successful_replacements.append(num_replacements)
        print(f"{sample[:32]:<35} {num_replacements:<12} {'✓ 成功':<12}")
    else:
        print(f"{sample[:32]:<35} {len(replacements):<12} {'✗ 失败':<12}")

print("-" * 70)
print(f"\n攻击成功率: {success_count}/{len(test_samples)} ({success_count/len(test_samples)*100:.1f}%)")

if success_count > 0:
    avg_replacements = total_replacements / success_count
    print(f"成功攻击的平均替换词数: {avg_replacements:.2f}")
    print(f"最少替换词数: {min(successful_replacements)}")
    print(f"最多替换词数: {max(successful_replacements)}")

print("=" * 70)

In [ ]:
# 可视化攻击前后的情感变化
print("\n可视化攻击效果...")

original_scores = []
adversarial_scores = []
labels_list = []
attack_success = []

for sample in test_samples:
    orig_score = get_positive_score(sample)
    adv_text, success, _ = synonym_attack(sample)
    adv_score = get_positive_score(adv_text)
    
    original_scores.append(orig_score)
    adversarial_scores.append(adv_score)
    labels_list.append(sample[:8] + "...")
    attack_success.append(success)

# 绘制对比图
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# 左图：置信度对比
x = np.arange(len(labels_list))
width = 0.35

bars1 = ax1.bar(x - width/2, original_scores, width, label='原始', color='steelblue', alpha=0.8)
bars2 = ax1.bar(x + width/2, adversarial_scores, width, label='攻击后', color='coral', alpha=0.8)

# 标记攻击成功的样本
for i, success in enumerate(attack_success):
    if success:
        ax1.plot([i - width/2, i + width/2], 
                [original_scores[i], adversarial_scores[i]], 
                'g--', linewidth=2, alpha=0.5)

ax1.axhline(y=0.5, color='gray', linestyle='--', linewidth=1.5, label='决策边界', alpha=0.7)
ax1.set_ylabel('正面情感置信度', fontsize=12)
ax1.set_title('同义词替换攻击：情感置信度变化', fontsize=13, fontweight='bold')
ax1.set_xticks(x)
ax1.set_xticklabels(labels_list, rotation=45, ha='right', fontsize=9)
ax1.legend(fontsize=10)
ax1.set_ylim(0, 1)
ax1.grid(axis='y', alpha=0.3)

# 右图：攻击成功率统计
success_counts = [sum(attack_success), len(attack_success) - sum(attack_success)]
colors = ['green', 'red']
labels = [f'成功 ({success_counts[0]})', f'失败 ({success_counts[1]})']

ax2.pie(success_counts, labels=labels, colors=colors, autopct='%1.1f%%', 
        startangle=90, textprops={'fontsize': 12})
ax2.set_title(f'攻击成功率统计\n总样本数: {len(test_samples)}', 
              fontsize=13, fontweight='bold')

plt.tight_layout()
plt.show()

## 第五部分：高级攻击技巧演示

除了同义词替换，还有其他文本攻击方法。

In [ ]:
### 技巧 1：同形字替换（视觉混淆）

# 定义同形字映射（外观相似但编码不同的字符）
SIMILAR_CHARS = {
    '好': '妤',   # 形近字
    '喜': '嘉',
    '优': '忧',
    '推': '推',   # 全角/半角
    '荐': '荐',
    '质': '貭',
    '量': '糧',
}

def similar_char_attack(text):
    """
    同形字替换攻击
    
    原理：使用外观相似的字符替换原字符
    特点：人眼难以察觉，但改变了字符编码
    """
    result = list(text)
    replacements = []
    
    for i, char in enumerate(result):
        if char in SIMILAR_CHARS:
            result[i] = SIMILAR_CHARS[char]
            replacements.append((char, SIMILAR_CHARS[char]))
    
    return ''.join(result), replacements

# 演示同形字攻击
test_text = "这个产品质量好，我很喜欢！"
similar_text, sim_replacements = similar_char_attack(test_text)

print("=" * 60)
print("🎭 同形字替换攻击演示")
print("=" * 60)
print(f"原始文本: {test_text}")
print(f"攻击文本: {similar_text}")
print(f"替换记录: {sim_replacements}")
print(f"\n原始分析: {analyze_sentiment(test_text)}")
print(f"攻击分析: {analyze_sentiment(similar_text)}")
print("\n💡 注意：两段文本看起来几乎一样，但字符编码不同")

In [ ]:
### 技巧 2：字符插入攻击（添加干扰）

def character_insertion_attack(text, insert_char='​'):
    """
    字符插入攻击
    
    原理：在词语中插入零宽字符或空格
    特点：不改变视觉效果，但影响分词
    
    参数:
        text: 原始文本
        insert_char: 要插入的字符（默认零宽空格）
    """
    # 在每个字之间插入零宽字符
    result = insert_char.join(list(text))
    return result

# ========== 填空 5：测试字符插入攻击 ==========
# 
# 🎯 任务：对测试文本执行字符插入攻击并观察效果
# 
# 💡 提示：
#   - 调用 character_insertion_attack 函数
#   - 参数是 test_text
#   - 观察对模型预测的影响
# 
# 难度：⭐☆☆☆☆

inserted_text = ___________  # 期望：character_insertion_attack(test_text)

print("\n" + "=" * 60)
print("🔤 字符插入攻击演示")
print("=" * 60)
print(f"原始文本: {test_text}")
print(f"插入后文本: {inserted_text}")
print(f"文本长度: {len(test_text)} → {len(inserted_text)}")
print(f"\n原始分析: {analyze_sentiment(test_text)}")
print(f"插入后分析: {analyze_sentiment(inserted_text)}")
print("\n💡 插入零宽字符可能干扰分词和模型理解")

## 📋 实验小结

### 核心收获

通过本实验，你已经：

1. **理解差异**：掌握了文本与图像对抗攻击的本质区别
2. **实现攻击**：成功实现了基于同义词替换的文本对抗攻击
3. **分析重要性**：学会了计算词重要性并设计针对性攻击
4. **探索技巧**：了解了多种文本攻击方法（同形字、字符插入等）

### 关键概念回顾

**文本对抗攻击的三大挑战**：

| 挑战 | 描述 | 解决方案 |
|------|------|---------|
| **离散性** | 文本由离散的词组成，不能微小改变 | 同义词替换、字符变换 |
| **语义保持** | 必须保持原文语义不变 | 使用同义词库、语言模型验证 |
| **语法正确** | 避免产生语法错误 | 基于规则的替换、语法检查 |

**攻击方法对比**：

| 方法 | 原理 | 优点 | 缺点 | 适用场景 |
|------|------|------|------|---------|
| **同义词替换** | 用同义词替换原词 | 保持语义，成功率高 | 需要同义词库 | 通用文本攻击 |
| **同形字替换** | 用外观相似的字符 | 人眼难察觉 | 可能改变语义 | 视觉混淆攻击 |
| **字符插入** | 插入零宽字符 | 不改变视觉 | 容易被过滤 | 绕过关键词检测 |
| **词序调整** | 改变词的顺序 | 简单直接 | 可能语法错误 | 句法不敏感模型 |

### 实验观察

根据实验结果，通常可观察到：
- **攻击成功率**：同义词替换在情感分析任务上的成功率通常在 60-80%
- **替换效率**：平均只需替换 1-3 个关键词就能改变预测
- **词重要性**：情感词（如"好"、"差"）的重要性远高于中性词
- **模型脆弱性**：即使是BERT这样的大模型也容易受到攻击

### 实际应用场景

文本对抗攻击的现实威胁：
- **垃圾邮件检测**：绕过垃圾邮件过滤器
- **内容审核**：规避敏感词检测
- **情感分析**：操纵评论情感判断
- **文本分类**：误导新闻分类系统

### 防御策略

1. **输入预处理**：
   - 字符归一化（全角→半角）
   - 去除特殊字符
   - 拼写检查

2. **对抗训练**：
   - 用对抗样本增强训练集
   - 提高模型对扰动的鲁棒性

3. **集成方法**：
   - 多个模型投票
   - 基于规则的验证

4. **检测机制**：
   - 语法检查
   - 语义一致性验证
   - 异常模式检测

### 填空参考答案

<details>
<summary>点击查看所有填空参考答案</summary>

**填空 1：计算删除该字后的文本**
```python
chars_without_i = chars[:i] + chars[i+1:]
```

**填空 2：创建替换后的文本**
```python
test_text = current_text.replace(word, synonym, 1)
```

**填空 3：执行同义词替换攻击**
```python
adv_text, success, replacements = synonym_attack(sentence)
```

**填空 4：批量测试并统计结果**
```python
adv_text, success, replacements = synonym_attack(sample)
```

**填空 5：测试字符插入攻击**
```python
inserted_text = character_insertion_attack(test_text)
```

</details>

### 延伸学习

- **论文阅读**：
  - "Adversarial Examples for Evaluating Reading Comprehension Systems" (EMNLP 2017)
  - "TextFooler: Is BERT Really Robust?" (AAAI 2020)
  - "BAE: BERT-based Adversarial Examples for Text Classification" (EMNLP 2020)

- **进阶挑战**：
  - 实现基于遗传算法的文本攻击
  - 使用语言模型生成更自然的对抗样本
  - 探索针对机器翻译、问答系统的攻击

- **工具推荐**：
  - TextAttack：文本对抗攻击框架
  - OpenAttack：开源文本对抗攻击库

### 🤔 深入思考

1. **伦理问题**：文本对抗攻击研究是否应该被限制？如何平衡研究和安全？
2. **实用性**：在哪些场景下，攻击者最有可能使用文本对抗攻击？
3. **未来方向**：大语言模型（如 GPT、Claude）是否更容易或更难被攻击？
4. **防御思路**：能否设计一个对文本对抗攻击完全鲁棒的模型？

### 安全提醒

⚠️ **本实验仅用于教育目的**

文本对抗攻击是评估NLP系统安全性的重要工具。请将所学知识用于：
- 测试和提升自己系统的鲁棒性
- 开发防御方法和检测机制
- 推进安全NLP研究

请勿用于：
- 绕过内容审核系统发布违规内容
- 制造虚假评论和恶意信息
- 攻击他人的服务和系统

### 🎉 模块三完成！

恭喜你完成了模块三的所有实验！你已经掌握了：
- ✅ 图像对抗攻击（FGSM, PGD）
- ✅ 迁移攻击（黑盒场景）
- ✅ 文本对抗攻击

继续学习后续模块，探索更多AI安全话题！